In [1]:
'''
Extracts RV data for the table in the paper.
Outputs the table
'''

'\nExtracts RV data for the table in the paper.\nOutputs the table\n'

In [4]:
import numpy as np
import csv
#from tabulate import tabulate
#from texttable import Texttable

MJUPITER = 1.898e27     #all in SI units
MEARTH = 5.972e24

In [5]:
class System:
    def __init__(self, sysid=None):
        
        self.sysid = sysid

In [6]:
#my_file = 'C:/Users/djhoo/Documents/Oviraptor-master/catalogues/oviraptor_crossmatch_catalog.csv'  #reading in file
my_file = "/Users/research/projects/oviraptor/Catalogs/old_oviraptor_crossmatch_catalog.csv"


raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[0]
VALUES = raw_data[1:]

def get_key(keyname, keys = KEYS, values = VALUES):
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return np.array(arr)

In [7]:
d = {}
d['starname'] = get_key('hostname')
d['rs'] = get_key('st_rad')
d['rs_lower'] = get_key('st_raderr2')
d['rs_upper'] = get_key('st_raderr1')
d['ms'] = get_key('st_mass')
d['ms_lower'] = get_key('st_masserr2')
d['ms_upper'] = get_key('st_masserr1')
d['disc_method'] = get_key('discoverymethod')
d['mag'] = get_key('sy_vmag')
d['temp'] = get_key('st_teff')
d['ref'] = get_key('st_refname')
d['instrument'] = get_key('disc_facility')
d['rp'] = get_key('pl_rade')
d['mp'] = get_key('pl_bmasse')
d['p'] = get_key('pl_orbper')

In [8]:
KEY0 = 'starname'

def is_number(s):
    '''
    checks if input string s is a number
    or if every entry in a list s is a number
    '''
    if type(s) != list:
        try:
            float(s)
            return True
        except ValueError:
            return False
    else:
        for part in s:
            status = True
            try:
                float(part)
                status = True
            except ValueError:
                status = False
        return status

def convert_to_arrays(d):
    keys = d.keys()
    dnew = {}
    
    for k in keys:
        dnew[k] = np.asarray(d[k])
        
    return dnew

d = convert_to_arrays(d)
print('total number of queried objects =', len(d[KEY0]))

for key in d.keys():
    if is_number(d[key][0]):
        for i in range(len(d[key])):
            if d[key][i] == '':
                d[key][i] = 0
            

def convert_floats(d):
    '''
    converts numerical arrays from strings to floats
    '''
    keys = d.keys()
    dnew = {}
    
    for k in keys:
        if is_number(d[k][0]):
            dnew[k] = np.asarray(d[k],dtype='float')
        else:
            dnew[k] = np.asarray(d[k])
    
    return dnew


# CONVERT STRINGS TO FLOATS (where applicable)
d = convert_floats(d)

total number of queried objects = 4107


In [9]:
def legitimate_periods(periods, detection_method, max_period):
    '''
    This function is used to eliminate planets in RV systems with too long periods.
    This is meant for RV planets, so planets detected by any other methods are ignored.
    '''
    entries = []
    for i in range(len(periods)):
        if periods[i] <= max_period and detection_method == 'Radial Velocity':
            entries.append(True)
        elif detection_method == 'Transit' or detection_method == 'Mixed':
            entries.append(True)
        else:
            entries.append(False)
    return np.array(entries)

def legitimate_radii(radii, detection_method, min_radius):
    '''
    This function is used to eliminate planets in transit systems with too high radii.
    This is meant for transit planets, so planets detected by any other methods are ignored.
    '''
    entries = []
    for i in range(len(radii)):
        if radii[i] >= min_radius and detection_method == 'Transit':
            entries.append(True)
        elif detection_method == 'Radial Velocity' or detection_method == 'Mixed':
            entries.append(True)
        else:
            entries.append(False)
    return np.array(entries)

max_period = 1460                                                       #length of time used to create DR25 catalogue
min_radius = min(d['rp'][d['disc_method'] == 'Radial Velocity'])        #minimum radius of an RV planet

uniquesys = np.unique(d['starname'])
systems = []
for us in uniquesys:
    s = System()                                               #I assume that each parameter is the same for all entries
    
    s.sysid = us
    s.starname = d['starname'][d['starname']==us][0]
    
    s.Mstar = d['ms'][d['starname']==us][0]
    s.Mstar_lower = d['ms_lower'][d['starname']==us][0]
    s.Mstar_upper = d['ms_upper'][d['starname']==us][0]
    
    s.Rstar = d['rs'][d['starname']==us][0]
    s.Rstar_lower = d['rs_lower'][d['starname']==us][0]
    s.Rstar_upper = d['rs_upper'][d['starname']==us][0]
    
    s.detection_method = d['disc_method'][d['starname']==us][0]
    s.mag = d['mag'][d['starname']==us][0]
    s.temp = d['temp'][d['starname']==us][0]
    s.ref = d['ref'][d['starname']==us][0]
    s.instrument = d['instrument'][d['starname']==us][0]
    
    s.masses = d['mp'][d['starname']==us]
    s.radii = d['rp'][d['starname']==us]
    s.periods = d['p'][d['starname']==us]
    
    legitimate_entries = legitimate_periods(s.periods, s.detection_method, max_period)
    s.masses = s.masses[legitimate_entries == True]
    s.radii = s.radii[legitimate_entries == True]
    s.periods = s.periods[legitimate_entries == True]
    
    legitimate_entries = legitimate_radii(s.radii, s.detection_method, min_radius)
    s.masses = s.masses[legitimate_entries == True]
    s.radii = s.radii[legitimate_entries == True]
    s.periods = s.periods[legitimate_entries == True]
    
    s.masses = s.masses[s.radii != 0]
    s.periods = s.periods[s.radii != 0]
    s.radii = s.radii[s.radii != 0]
    
    s.periods = s.periods[s.masses*MEARTH/MJUPITER < 13]
    s.radii = s.radii[s.masses*MEARTH/MJUPITER < 13]
    s.masses = s.masses[s.masses*MEARTH/MJUPITER < 13]
    
    if len(s.masses) >= 3 and s.detection_method == 'Radial Velocity':
        systems.append(s)

/Users/research/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/research/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/research/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
print(len(systems))

# Generates Latex table
# Need to implement formatting the reference

rows = []
for s in systems:
    row = [0]*11
    row[0] = s.starname
    row[1] = s.Mstar
    row[2] = s.Mstar_upper
    row[3] = s.Mstar_lower
    row[4] = s.Rstar
    row[5] = s.Rstar_upper
    row[6] = s.Rstar_lower
    row[7] = s.mag
    row[8] = s.temp
    row[9] = s.instrument
    row[10] = s.ref[s.ref.index('r=')+2:s.ref.index('href')]
    rows.append(row)

In [ ]:
print('\nTabulate Latex:')
print(tabulate(rows, headers='firstrow', tablefmt='latex'))